### [실습] PDF 기반 질의응답 시스템 구축


이 코드는 하나의 질문에 대한 답을 찾거나, 긴 문서를 짧고 쉽게 요약하는 "스마트한 독서 비서"를 만드는 프로젝트입니다.
우리가 PDF 파일을 던져주면, AI가 그 내용을 읽고 이해한 뒤, 우리가 던지는 질문에 대답하거나 전체 내용을 요약해주는 구조입니다.

🏗️ 1. 전체 흐름을 '큰 그림'으로 먼저 보기
💡 비유로 이해하기:

"AI 비서를 훈련시켜 책 한 권을 읽히고, 요약도 잘하게 만들고, 궁금한 것도 물어보게 하는 시스템"을 만드는 작업입니다.



📌 전체 과정은 다음 3단계입니다:
📄 PDF 문서 불러오기 + 읽을 수 있는 상태로 전처리

책을 스캔해서 페이지마다 내용을 꺼내오는 단계

🧠 문서의 내용들을 AI가 이해할 수 있도록 '기억'시키기 (벡터화)

내용을 AI의 뇌 속에 집어넣기 (숫자로 바꿔 기억하게 함)

❓ 질문하거나 📚 요약 요청하기

사용자가 질문을 던지면 관련 내용을 찾아 AI가 답변

또는 요약 요청하면 전체 내용을 간단히 정리해줌



🔍 2. 주요 기능 2가지
✅ 기능 1: 질의응답 시스템
AI가 문서 전체를 읽고 기억한 뒤, "이 문서에서 핵심 주제가 뭐야?" 같은 질문에 답해줍니다.

사용자 질문에 가장 적합한 부분을 찾아서 답변을 생성합니다.

✅ 기능 2: 문서 요약 시스템
너무 긴 문서를 짧고 핵심만 남긴 요약문으로 줄여줍니다.

요약 방식은 총 3가지가 있습니다:

Stuff 방식: 문서 전체를 한 번에 요약

Refine 방식: 요약을 반복적으로 다듬어가며 완성

Map-Reduce 방식: 조각조각 요약 후 통합 요약



📌 3. 왜 이런 방식이 필요할까?
단순히 "문서 읽기"로는 부족합니다. 문서가 길거나 복잡하면,

어디에 뭐가 있는지 찾기 힘들고

전체 내용을 요약하기 어려운 문제가 생깁니다.

그래서 필요한 것이 바로:

정보를 AI가 '벡터'라는 형태로 기억하도록 가공하는 과정

그 기억 속에서 원하는 정보를 뽑아내는 능력



🧩 4. 주요 기술 개념 요약
기술 용어	쉬운 설명
벡터화	문장의 의미를 숫자로 표현해 AI가 기억하게 함
RAG	질문에 맞는 문서를 먼저 찾아 답변에 활용하는 방식
LLM	질문에 대해 자연스럽게 답하는 대형 언어 모델
체인(chain)	여러 AI 작업을 연결한 파이프라인처럼 생각하면 됨



🗂️ 5. 요약하자면...
PDF → 텍스트로 변환 → AI가 이해하도록 변환(벡터화)

사용자는 질문하거나 요약을 요청

AI는 관련 내용을 찾아 응답하거나 요약해줌

✨ 우리가 만든 것은 AI 독서 도우미입니다!



In [ ]:
# ✅ [필수 라이브러리 설치]
# 이 코드에서는 OpenAI, LangChain, ChromaDB 등을 활용해 PDF 문서 기반 질의응답 시스템을 구축합니다.
# 아래는 관련된 패키지를 설치하는 명령어입니다.
!pip install --upgrade openai langchain langchain-openai langchain-community -q
!pip install chromadb tiktoken -q
!pip install pymupdf pypdf pypdf2 -q

In [ ]:
# ✅ [필수 모듈 불러오기]
# os: 환경변수 설정에 사용
# re: 정규표현식을 활용한 문자열 전처리용
import os
import re

# ✅ OpenAI 임베딩 모델과 ChromaDB 벡터스토어를 불러옵니다.
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# ✅ PDF 파일을 로드하기 위한 문서 로더 불러오기
from langchain_community.document_loaders import PyMuPDFLoader

# ----------------------------------------
# ✅ 1. PDF 문서 로드 및 전처리 단계
# ----------------------------------------

# ✅ [포인트] OpenAI API 키 설정
# OpenAI 서비스를 사용하기 위해 개인 API 키를 환경변수에 설정합니다.
# (강의 시에는 키를 직접 노출하지 말고 보안에 유의해야 합니다!)

# ✅ PDF 문서 경로를 설정합니다. (로컬 경로 또는 Colab 경로 사용 가능)
pdf_path = '/content/test.pdf'

# ✅ PDF 문서를 불러와 페이지 단위로 내용을 나눕니다.
loader = PyMuPDFLoader(pdf_path)
pages = loader.load()

# ✅ 전처리 코드 예시 (주석 처리됨)
# 특수문자, 불필요한 공백 등을 정리할 때 사용합니다.
# page.page_content는 페이지의 텍스트 내용입니다.
# for page in pages:
#     page.page_content = re.sub(r'[^\w\s]', '', page.page_content.replace('\xa0', '').replace('\n\n', '\n'))

# ✅ 확인용으로 특정 페이지 출력 (예: 4번째 페이지)
print(pages[3])


In [ ]:
# ----------------------------------------
# ✅ 2. 질의응답 시스템 구성 (Prompt + LLM)
# ----------------------------------------

# ✅ ChatGPT 모델을 불러오는 모듈과 체인을 구성하기 위한 컴포넌트 불러오기
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# ✅ 시스템 지침 프롬프트 구성
# - 모델에게 "질문에 답할 때 문서 내용을 기반으로 판단하라"고 지시합니다.
# - 'summaries'라는 문서 내용을 입력값으로 받아 사용하게 됩니다.
system_prompt = """'질의응답' 과정의 자료가 아래에 주어집니다. 질문에 대해 아래의 순서로 답변하세요. 한국어로 답변하세요.
1. 주어진 문서의 내용이 질문에 얼마나 적합한지 평가하세요.
2. 질문에 대한 답변을 출력하세요.
---
{summaries}
---
Answer:"""

# ✅ 프롬프트 템플릿 설정
# SystemMessagePromptTemplate: 시스템 메시지 (지침)
# HumanMessagePromptTemplate: 사용자 질문 입력
messages = [
    SystemMessagePromptTemplate.from_template(system_prompt),
    HumanMessagePromptTemplate.from_template("Question: {question}")
]

# ✅ 위에서 정의한 메시지들을 하나의 프롬프트 체인으로 결합
prompt = ChatPromptTemplate.from_messages(messages)

### 📘 문서 내용을 임베딩하고 벡터 데이터베이스에 저장하는 과정

이번 단계에서는 우리가 PDF에서 불러온 문서 내용을 **임베딩(Embedding)**이라는 과정을 통해 수치화하고, 그 결과를 **Chroma라는 벡터 데이터베이스**에 저장합니다. 이 작업은 추후 질문을 했을 때 GPT가 문서 내용을 기반으로 정확히 답변할 수 있도록 도와주는 핵심적인 준비 작업입니다.

---

#### ✅ 임베딩이란 무엇인가요?

임베딩이란, 우리가 사용하는 자연어(예: "AI는 미래다")를 **컴퓨터가 이해할 수 있는 숫자 벡터 형태**로 바꾸는 작업입니다.  
예를 들어, 문장 하나가 다음처럼 바뀝니다:

> "AI는 미래다" → [0.42, -0.15, 0.87, 1.03, ...]

이런 숫자 벡터는 문장의 **의미와 뉘앙스를 담고 있어서**, 유사한 문장끼리는 비슷한 벡터를 갖게 됩니다.  
즉, 단순한 문자 비교가 아니라 **"의미 기반 검색"이 가능**해지는 것이죠.

---

#### ✅ 왜 임베딩을 할까요?

사람은 글자를 보고 의미를 파악하지만, GPT 같은 AI 모델은 **숫자 형태의 데이터**만 처리할 수 있습니다.  
그래서 우리는 문서의 내용을 숫자 벡터로 바꾸어야 합니다. 그리고 나서 그 벡터들을 모아놓을 장소가 바로 **벡터 데이터베이스**입니다.

---

In [ ]:
# ----------------------------------------
# ✅ 3. 문서 내용 임베딩 및 벡터 DB 구축
# ----------------------------------------

# ✅ [포인트] 텍스트 임베딩: 텍스트를 수치 벡터로 변환하는 과정
# 예시: "AI는 미래다" → [0.34, -0.12, ...]
# 이렇게 벡터화된 텍스트를 DB에 저장하면 유사한 내용을 빠르게 검색할 수 있습니다.
embeddings = OpenAIEmbeddings()

# ✅ 기존 ChromaDB 컬렉션(데이터베이스)을 초기화 (필수는 아님, 새로 구축 시 유용)
Chroma().delete_collection()

# ✅ 페이지 내용을 임베딩하여 ChromaDB에 저장합니다.
db = Chroma.from_documents(pages, embeddings)

# ✅ [포인트] 검색기(retriever) 설정: 관련된 문서를 k=5개까지 검색
retriever = db.as_retriever(search_kwargs={"k": 5})

In [ ]:
# ----------------------------------------
# ✅ 4. LLM + 체인 구성
# ----------------------------------------

# ✅ OpenAI GPT-4o 모델을 사용하도록 설정
# temperature=0.1: 출력의 창의성 낮추기 → 더 일관된 답변
# max_tokens=4096: 출력 가능한 최대 토큰 수 설정
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.1, max_tokens=4096)

# ✅ 전체 체인 구성
# 순서:
# 1. 사용자의 질문이 들어오면
# 2. retriever가 관련된 문서 내용을 찾아 'summaries'에 넣고
# 3. prompt를 통해 지침과 함께 LLM에게 전달
# 4. LLM이 최종 답변을 생성
# 5. StrOutputParser()는 텍스트 결과만 추출
chain = (
    {"summaries": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# ----------------------------------------
# ✅ 5. 실제 질의응답 실행
# ----------------------------------------

# ✅ 사용자 질문 정의
query = "이 문서에서 다루는 주요 주제는 무엇인가요?"

# ✅ 체인을 실행하여 답변 생성
answer = chain.invoke(query)

# ✅ 최종 결과 출력
print(answer)

1. 주어진 문서의 내용이 질문에 적합합니다. 문서는 여자 게일식 축구의 경기 규칙에 대해 설명하고 있습니다.

2. 이 문서에서 다루는 주요 주제는 여자 게일식 축구의 경기 규칙입니다. 문서는 경기 시간, 선수의 장비, 팀 구성 등 다양한 규칙을 설명하고 있습니다.


# [실습] PDF 문서 요약

### 1. Stuff 방식 요약


In [ ]:
# 1. Stuff 방식 요약
# Stuff 방식은 전체 문서를 한 번에 요약하는 방법입니다. 즉, 모든 문서를 한 번에 입력으로 받아 요약을 생성합니다.

# 필요한 모듈을 불러옵니다.
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain

# 새로운 Document 객체를 생성합니다. 이 객체는 여러 페이지의 내용을 하나로 합친 코퍼스(corpus)를 의미합니다.
corpus = Document(page_content='')

# 주어진 페이지(pages) 리스트에서 각 페이지의 내용을 하나의 문자열로 합칩니다.
# 각 페이지의 내용을 하나로 연결하고, 각 페이지 사이에 줄바꿈('\n')을 추가합니다.
for page in pages:
    corpus.page_content += page.page_content + '\n'

# 문서 내에 여러 개의 줄바꿈이 있을 경우, 이를 하나의 줄바꿈으로 변경하여 정리합니다.
corpus.page_content = corpus.page_content.replace('\n\n', '\n')

# 문서를 청크(chunk) 단위로 나눕니다.
# chunk_size는 각 청크의 최대 길이를 설정하며, chunk_overlap은 각 청크 간의 중복 길이를 설정합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)

# 문서를 청크 단위로 분할합니다.
# 문서 전체(corpus)를 리스트로 감싸 전달하여, 이를 청크로 나눈 리스트(chunks)를 생성합니다.
chunks = text_splitter.split_documents([corpus])



# 요약을 위한 프롬프트 템플릿을 정의합니다.
# 이 템플릿은 주어진 텍스트를 받아, 2개의 문장으로 구성된 1개의 문단으로 요약하라는 지시를 포함하고 있습니다.
stuff_prompt_template = """주어진 문서의 내용을 요약하세요.
요약은 1개의 문단으로 구성되며, 각 문단은 2개의 문장으로 작성하세요.
---
{text}
---
요약:"""

# 위에서 정의한 템플릿을 기반으로 실제 사용할 프롬프트 객체를 생성합니다.
stuff_prompt = PromptTemplate.from_template(stuff_prompt_template)

# 요약 체인을 로드합니다.
# 체인 유형으로 "stuff"를 선택하고, 앞서 정의한 프롬프트를 사용합니다.
stuff_chain = load_summarize_chain(llm, chain_type="stuff", prompt=stuff_prompt)

# 요약 체인을 실행하여 요약 결과를 얻습니다.
# chunks는 위에서 분할한 청크들의 리스트입니다.
# 'invoke' 메서드를 사용하여 요약 작업을 수행하고, 결과를 stuff_summary에 저장합니다.
stuff_summary = stuff_chain.invoke({"input_documents": chunks})

# 요약된 텍스트를 출력합니다.
print(stuff_summary["output_text"])


이 문서는 2022년 여자 게일식 축구의 경기 규칙을 간략히 설명한 소책자로, 선수와 관계자들이 경기 규칙을 이해하도록 돕는 것을 목표로 합니다. 경기장 크기, 선수 복장, 경기 시간, 팀 구성, 교체 규칙, 출혈 및 두부 손상 의심 규정 등 다양한 규칙이 포함되어 있으며, 모든 규칙은 공식 가이드에서 확인할 수 있습니다.


# 2. Refine 방식 요약


In [ ]:
# 2. Refine 방식 요약
# 이 요약 방법은 긴 텍스트를 여러 부분(청크)으로 나눠 단계별로 요약하는 방식입니다.
# 먼저 첫 번째 청크를 요약하고, 그 후 나머지 청크들을 하나씩 추가하면서 기존 요약을 보완해 나갑니다.
# 최종적으로 모든 청크를 통합하여 완성된 요약이 출력됩니다.
# 요약은 처음 요약과 새로운 청크를 기반으로 반복적으로 업데이트됩니다.

# 초기 요약을 위한 프롬프트 템플릿을 정의합니다.
# 주어진 텍스트를 2개의 문단과 각 문단별 5개의 문장으로 요약하도록 지시합니다.
initial_prompt_template = """주어진 텍스트를 2개의 문단과 각 문단별 5개의 문장으로 요약하세요."""

# 첫 번째 청크에 대해 초기 요약을 수행합니다.
# 'chunks[0].page_content'는 첫 번째 청크의 내용을 의미하며, 이를 요약합니다.
initial_summary = llm.invoke(
    PromptTemplate.from_template(initial_prompt_template).format(text=chunks[0].page_content)
).content

# 추가적인 요약 보완을 위한 프롬프트 템플릿을 정의합니다.
# 현재까지 요약된 내용을 참고하여, 새로운 청크의 내용을 추가하고 요약을 보완하도록 지시합니다.
refine_prompt_template = """현재 요약된 내용을 참고하여 새로운 내용을 추가하고, 요약을 보완하세요.
---
현재 요약: {previous_summary}
새로운 내용: {new_text}"""

# 첫 번째 청크 이후의 나머지 청크들을 반복하여 처리합니다.
# 각 청크마다 현재까지의 요약을 업데이트합니다.
for i in range(1, len(chunks)):
    refine_response = llm.invoke(
        PromptTemplate.from_template(refine_prompt_template).format(
            previous_summary=initial_summary,
            new_text=chunks[i].page_content
        )
    )
    # 새로 보완된 요약을 'initial_summary'에 저장합니다.
    initial_summary = refine_response.content

# 최종 요약 결과를 출력합니다.
print(initial_summary)


보완된 요약: 주어진 텍스트는 축구 경기 규칙에 대한 내용을 다루고 있습니다. 주요 내용은 다음과 같습니다:

1. **축구공 규격**:
   - 4호 축구공은 U-12부터 모든 등급의 경기에 사용되며, 더 작은 3호 공은 어린 연령대에서 사용됩니다.

2. **선수 등번호**:
   - 각 선수의 등번호는 팀 명단과 일치해야 합니다.

3. **팀 명단**:
   - 팀 명단이 15명을 초과할 경우, 첫 15명이 실제 팀을 구성하는 것으로 간주됩니다.
   - 클럽, 카운티 또는 주에서는 선발 선수 15명 및 교체 선수 명단을 조직위원회에 제공해야 합니다.

4. **교체 및 연장 시간 규정**:
   - 내셔널 리그 8강까지와 프로빈셜 유소년 챔피언십 조별 리그에서는 무제한 교체가 허용됩니다.
   - 내셔널 리그 8강 이후와 성인 프로빈셜 및 내셔널 챔피언십에서는 5명의 교체만 허용됩니다.
   - 연장 시간에는 추가로 5명의 교체가 허용됩니다.
   - 카운티 내 클럽 수준에서는 무제한 교체가 가능하며, 교체 횟수는 카운티 이사회에서 결정합니다.

5. **퇴장 규정**:
   - 경기 중 퇴장당한 선수는 교체될 수 없으나, 연장 시간에는 교체가 가능합니다.

6. **출혈 및 두부 손상 의심 규정**:
   - 출혈이나 두부 손상이 의심되는 선수는 즉시 경기장을 떠나 치료를 받아야 합니다.
   - 출혈이 멎고 깨끗해진 후에야 경기장에 복귀할 수 있습니다.
   - 출혈/두부 손상 의심 교체 선수는 정규 교체로 간주되지 않습니다.
   - 출혈/두부 손상 의심 선수가 퇴장당했을 경우, 다른 선수로 교체해야 하며 이는 정규 교체로 간주되지 않습니다.
   - 정규 교체를 모두 사용한 경우에도 출혈/두부 손상 의심 선수로 교체될 수 있습니다.
   - 출혈/두부 부상 의심 선수가 경기 재개 준비가 되었을 때, 주심의 판단에 따라 경기장에 복귀할 수 있습니다.
   - 원래 선수를 다시 출전시키기를 원할 때 다른 선수로 교체해야 하며, 이는 정규 교체로 간

# 3. Map-Reduce 방식 요약


In [ ]:
# 3. Map-Reduce 방식 요약
# 문서를 여러 청크로 나눈 후, 각각의 청크를 요약(Map 과정)하고, 그 요약들을 합쳐 최종 요약을 생성합니다(Reduce 과정).
# 각 청크를 요약하기 위한 프롬프트 템플릿을 정의합니다.
# 텍스트를 1-3개의 문단과 각 문단별 5개의 문장으로 요약하도록 지시합니다.
map_prompt_template = """주어진 텍스트를 1-3개의 문단과 각 문단별 5개의 문장으로 요약하세요."""\

map_prompt = PromptTemplate.from_template(map_prompt_template)
# Map 과정: 각 청크에 대해 개별적으로 요약을 생성합니다.
map_summaries = []  # 각 청크의 요약을 저장할 리스트입니다.
for chunk in chunks:
    # 각 청크에 대해 요약을 생성하고, 결과를 리스트에 추가합니다.
    map_response = llm.invoke(map_prompt.format(text=chunk.page_content))
    map_summaries.append(map_response.content)

# Reduce 과정: 생성된 모든 요약을 합쳐 최종 요약을 만듭니다.
# 여러 개의 요약을 바탕으로 전체 문서의 내용을 요약하도록 지시하는 프롬프트 템플릿을 정의합니다.
reduce_prompt_template = """주어진 요약들을 바탕으로 전체 내용을 요약하세요.
요약은 5개의 문단으로 구성되며, 각 문단은 4-8개의 문장으로 작성하세요."""
reduce_prompt = PromptTemplate.from_template(reduce_prompt_template)

# 앞서 생성된 개별 요약들을 하나의 텍스트로 합친 후, 최종 요약을 생성합니다.
reduce_summary = llm.invoke(reduce_prompt.format(text='\n---\n'.join(map_summaries)))

# 최종 요약 결과를 출력합니다.
print(reduce_summary.content)


물론입니다. 주어진 요약들을 바탕으로 전체 내용을 요약해드리겠습니다.

첫 번째 문단에서는 주제와 배경을 소개합니다. 이 문단에서는 주요 인물, 사건, 또는 주제를 간략하게 설명하며 독자가 전체 내용을 이해하는 데 필요한 기본 정보를 제공합니다. 예를 들어, 이야기의 배경이 되는 시간과 장소, 주요 인물들의 관계, 그리고 사건의 발단 등을 포함할 수 있습니다.

두 번째 문단에서는 주요 사건의 전개를 다룹니다. 여기서는 이야기의 중요한 전환점이나 갈등을 설명합니다. 주요 인물들이 어떤 행동을 취했는지, 그로 인해 어떤 결과가 발생했는지 등을 구체적으로 서술합니다. 이 문단은 독자가 이야기의 흐름을 따라가며 긴장감을 느낄 수 있도록 구성합니다.

세 번째 문단에서는 갈등의 절정을 다룹니다. 이 부분에서는 이야기의 가장 극적인 순간이나 중요한 결정을 설명합니다. 주요 인물들이 직면한 도전과 그들이 선택한 해결 방법을 강조합니다. 이 문단은 독자가 이야기의 클라이맥스를 이해하고 감정적으로 몰입할 수 있도록 돕습니다.

네 번째 문단에서는 갈등의 해결과 그 결과를 설명합니다. 여기서는 주요 사건이 어떻게 마무리되었는지, 그리고 그로 인해 인물들이 어떤 변화를 겪었는지를 다룹니다. 또한, 이야기의 결말이 독자에게 어떤 메시지를 전달하는지 설명할 수 있습니다. 이 문단은 이야기의 여운을 남기며 독자가 전체적인 결말을 이해하도록 돕습니다.

마지막 문단에서는 전체 내용을 요약하고, 이야기의 주제나 교훈을 강조합니다. 이 문단에서는 이야기의 핵심 메시지를 다시 한 번 상기시키며, 독자가 기억해야 할 중요한 점들을 정리합니다. 또한, 이야기의 여운을 남기며 독자가 생각할 거리를 제공할 수 있습니다. 이로써 전체 요약이 완성됩니다.
